# Matrix of metrics of Cross validation  for 5 top Simple Combination 
Date: 09/12/2022
Description: Create cross validation metrics
Use 5 combinations only regarding to the ranking from simple combination result.
This file creates cross validation metrics for next steps
Modificaiton 02/01/2023
Sử dụng 5 combination mới

In [1]:
!nvidia-smi

Fri Feb 17 12:44:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 6000     On   | 00000000:21:00.0 Off |                  Off |
| 34%   61C    P2   111W / 260W |   3255MiB / 24576MiB |     83%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Install packages
# !pip install -U scikit-learn
# !pip install ftfy
# !pip install xlsxwriter 
# !pip install xlwt

In [3]:
# Imports
import tensorflow as tf 
# from google.colab import drive
import os

from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense

import traceback
import sys
import copy
import numpy as np
from itertools import combinations
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor

# lib for LSTM of Meta model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler
import seaborn as sns


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable

from ftfy import fix_text
import warnings
warnings.simplefilter("ignore", UserWarning)

plt.rcParams['figure.figsize'] = [7, 3]
import math

# Global param and project path
All mix data 1997-2018

In [4]:
# Attach Google Drive for reading and saving files
# drive.mount('/content/drive')
prj_path = '/home/mlworker/Quang/HealthCare/Source_14012023_v4/'

# Lấy từ 1997 tới 01/01/2018. Bỏ 12 tháng cuối rồi
path_all_mix_data =             prj_path+ 'results/all_mix_data/all_mix_data_1997_2018_mae/'
path_all_mix_data_cnn =         prj_path+ 'results/all_mix_data/all_mix_data_1997_2018_mae/cnn/'
path_all_mix_data_lstm =        prj_path+ 'results/all_mix_data/all_mix_data_1997_2018_mae/lstm/'
path_all_mix_data_lstm_att =    prj_path+ 'results/all_mix_data/all_mix_data_1997_2018_mae/lstm_att/'
path_all_mix_data_tf =          prj_path+ 'results/all_mix_data/all_mix_data_1997_2018_mae/tf/'

prj_path_opt= prj_path+ 'optimize_hyperparam/opt_results/opt_results_1997_2018_mae/'


prj_path_result_ensemble =      prj_path+ 'results/ensemble_17022023/'
com_path_validation_metrics =   prj_path_result_ensemble+ 'result/weighted_combination/cv_metrics/'

# com_eval_path_validation_metrics = 'H:/Store/Research/UIT/PhD Research/Dev Environment/Tropical Diseases/running/eval_result/21_12_2022_weighted_combination-5-cities/'


# loockback_window for meta model
meta_lookback_window = 3

# Xây dựng Meta Data Set

In [5]:
##########################
# Xây Dựng Meta Data Set
meta_test_set_city = {}
meta_train_set_city = {}
meta_data_set_city = {}
cities = [
#     'An Giang', 'BR Vũng Tàu', 'Bình Phước', 'Bình Thuận', 'Bình Định',
#         'Bạc Liêu', 'Bắc Kạn', 'Bắc Giang', 
#     'Cao Bằng', 'Cà Mau', 
    
#         'Cần Thơ', 'Gia Lai', 'Hà Giang', 'Hà Nội', 'Hà Tĩnh', 'Hòa Bình',
#         'Hưng Yên', 'Hải Dương', 'Hải Phòng', 
#     'Khánh Hòa', 'Kiên Giang',
    
        'Kon Tum', 'Lai Châu', 'Long An', 'Lào Cai', 'Lâm Đồng',
        'Lạng Sơn', 'Nam Định', 'Nghệ An', 
#     'Ninh Bình', 'Ninh Thuận',
    
#         'Phú Thọ', 'Phú Yên', 'Quảng Bình', 'Quảng Nam', 'Quảng Ngãi',
#         'Quảng Ninh', 'Quảng Trị', 'Sóc Trăng',
#     'Sơn La', 'TT Huế',
    
#         'Thanh Hóa', 'Thái Bình', 'Thái Nguyên', 'Tiền Giang', 'Trà Vinh',
#         'Tuyên Quang', 'Tây Ninh', 'Vĩnh Phúc', 
#     'Yên Bái', 'Điện Biên',
    
#          'Đà Nẵng', 'Đắk Nông', 'Đắk Lắk', 'Đồng Tháp'
]

for city in cities:  
  meta_test_set = pd.DataFrame()
  meta_train_set = pd.DataFrame() 
  #Note: mỗi bộ all_mix_data sẽ có lenght = 251 (1997 -01/01/2018  và vì 1 dòng đầu bị bỏ đi do quá trình statitionary và remove NAN lúc train)
  lenght_whole_data_city = 251
  ######################################
  # Xử lý data của CNN ứng với mỗi city
  df = pd.read_excel(path_all_mix_data_cnn+'mix_train_test_data_'+city+'_cnn.xlsx')
  selected_result = df.sort_values(by = 'rmse', ascending=True) # Sắp xếp theo rmse nhỏ nhất
  selected_result = selected_result.head(lenght_whole_data_city) # lấy 251 phần tử đầu tiên
  selected_result= selected_result.sort_index() 
  # Tạo meta testset
  result_test = selected_result['Train Prediction'].iloc[-meta_lookback_window-36:-36] # Lấy buffer cho phần lookback_window
  result_test = result_test.append(selected_result['Test Prediction'].tail(36))
  meta_test_cnn = result_test.reset_index(drop=True)  
  # Tạo meta trainset
  result_train = selected_result['Train Prediction'].iloc[meta_lookback_window:-36]
  meta_train_cnn = result_train.reset_index(drop=True)
  
  ######################################
  # Xử lý data của LSTM  ứng với mỗi city
  df = pd.read_excel(path_all_mix_data_lstm+'mix_train_test_data_'+city+'_lstm.xlsx')  
  selected_result = df.sort_values(by = 'rmse', ascending=True) # Sắp xếp theo rmse nhỏ nhất
  selected_result = selected_result.head(lenght_whole_data_city) # lấy 251 phần tử đầu tiên
  selected_result= selected_result.sort_index()   
  
  # Tạo meta testset
  result_test = selected_result['Train Prediction'].iloc[-meta_lookback_window-36:-36] # Lấy buffer cho phần lookback_window
  result_test = result_test.append(selected_result['Test Prediction'].tail(36))
  meta_test_lstm = result_test.reset_index(drop=True)
  # Tạo meta trainset
  result_train = selected_result['Train Prediction'].iloc[meta_lookback_window:-36]
  meta_train_lstm = result_train.reset_index(drop=True)

  ###########################################
  # Xử lý data của LSTM ATT ứng với mỗi city
  df = pd.read_excel(path_all_mix_data_lstm_att+'mix_train_test_data_'+city+'_lstm_att.xlsx') 
  selected_result = df.sort_values(by = 'rmse', ascending=True) # Sắp xếp theo rmse nhỏ nhất
  selected_result = selected_result.head(lenght_whole_data_city) # lấy 251 phần tử đầu tiên
  selected_result= selected_result.sort_index()   
  
  # Tạo meta testset
  # Meta testset sẽ có lenght = 36 + lookback_window 
  meta_test_observed = selected_result['Observed'].iloc[-meta_lookback_window-36:]
  meta_test_observed = meta_test_observed.reset_index(drop=True) 

  result_test = selected_result['Train Prediction'].iloc[-meta_lookback_window-36:-36] # Lấy buffer cho phần lookback_window
  result_test = result_test.append(selected_result['Test Prediction'].tail(36))
  meta_test_lstm_att = result_test.reset_index(drop=True)  
  
  # Tạo meta trainset
  meta_train_observed = selected_result['Observed'].iloc[meta_lookback_window:-36]
  meta_train_observed = meta_train_observed.reset_index(drop=True)
  result_train = selected_result['Train Prediction'].iloc[meta_lookback_window:-36]
  meta_train_lstm_att = result_train.reset_index(drop=True)


  #############################################
  # Xử lý data của Transformer ứng với mỗi city
  df = pd.read_excel(path_all_mix_data_tf+'mix_train_test_data_'+city+'_tf.xlsx') 
  selected_result = df.sort_values(by = 'rmse', ascending=True) # Sắp xếp theo rmse nhỏ nhất
  selected_result = selected_result.head(lenght_whole_data_city) # lấy 251 phần tử đầu tiên
  selected_result= selected_result.sort_index()   
  
  # Tạo meta testset
  # Meta testset sẽ có lenght = 36 + lookback_window 
  meta_test_observed = selected_result['Observed'].iloc[-meta_lookback_window-36:]
  meta_test_observed = meta_test_observed.reset_index(drop=True) 

  result_test = selected_result['Train Prediction'].iloc[-meta_lookback_window-36:-36] # Lấy buffer cho phần lookback_window
  result_test = result_test.append(selected_result['Test Prediction'].tail(36)) # Lấy 36 tháng cuối
  meta_test_tf = result_test.reset_index(drop=True)  
  
  # Tạo meta trainset
  meta_train_observed = selected_result['Observed'].iloc[meta_lookback_window:-36] # Lấy từ meta_lookback_window cho đến -36 (3 năm cuối)
  meta_train_observed = meta_train_observed.reset_index(drop=True)
  result_train = selected_result['Train Prediction'].iloc[meta_lookback_window:-36]
  meta_train_tf = result_train.reset_index(drop=True)

  
  meta_test_set['cnn_pred'] = meta_test_cnn
  meta_test_set['lstm_pred'] = meta_test_lstm
  meta_test_set['lstm_att_pred'] = meta_test_lstm_att
  meta_test_set['tf_pred'] = meta_test_tf
  meta_test_set['Observed'] = meta_test_observed
   
  meta_train_set['cnn_pred'] = meta_train_cnn
  meta_train_set['lstm_pred'] = meta_train_lstm
  meta_train_set['lstm_att_pred'] = meta_train_lstm_att 
  meta_train_set['tf_pred'] = meta_train_tf  
  meta_train_set['Observed'] = meta_train_observed

  meta_test_set_city[city] = meta_test_set # test set có chiều dài 39
  meta_train_set_city[city] = meta_train_set # train set có chiều dài 212 = 252 -36 (train) - 1 (station) - 3 (meta_lookback_window)

  # Đoạn này làm thêm. Nối dài meta_test_set_city và meta_train_set_city
  # lưu ý đoạn này sẽ tạo ra data có chiều dài = 252 - 1 (bỏ đi 1 tháng đầu do xử lý stationary) - meta_lookback_window
  df = meta_train_set.append(meta_test_set[3:])
  df = df.reset_index(drop=True)
  # range date từ 1 + lookback_window tháng đầu tiên
  df['year_month'] = pd.Series(pd.date_range("1997-05-01", periods=248, freq="M"))  
  meta_data_set_city[city] = df
  df.to_excel(prj_path_result_ensemble+'/meta_data/meta_data_'+city+'.xlsx')

<ipython-input-5-7ab223f2f5fb>:43: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_test = result_test.append(selected_result['Test Prediction'].tail(36))
<ipython-input-5-7ab223f2f5fb>:58: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_test = result_test.append(selected_result['Test Prediction'].tail(36))
<ipython-input-5-7ab223f2f5fb>:77: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_test = result_test.append(selected_result['Test Prediction'].tail(36))
<ipython-input-5-7ab223f2f5fb>:100: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_test = result_test.append(selected_result['Test Prediction'].tail(36)) # L

<ipython-input-5-7ab223f2f5fb>:43: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_test = result_test.append(selected_result['Test Prediction'].tail(36))
<ipython-input-5-7ab223f2f5fb>:58: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_test = result_test.append(selected_result['Test Prediction'].tail(36))
<ipython-input-5-7ab223f2f5fb>:77: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_test = result_test.append(selected_result['Test Prediction'].tail(36))
<ipython-input-5-7ab223f2f5fb>:100: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_test = result_test.append(selected_result['Test Prediction'].tail(36)) # L

In [6]:
# meta_train_set_city['Lạng Sơn']

In [7]:
# meta_test_set_city['Lạng Sơn']

# Danh sách các based Model và các Combination có thể của nó

In [8]:
# Danh sách các based Model và các Combination có thể của nó
list_based_model = ['cnn_pred','lstm_att_pred','lstm_pred','tf_pred']

combs_features = list(combinations(list_based_model[0:], 2)) + list(combinations(list_based_model[0:], 3)) + list(combinations(list_based_model[0:], 4))

# list_based_model = ['cnn_pred','lstm_att_pred','lstm_pred','tf_pred']

# combs_features = list(combinations(list_based_model[0:], 3)) 
combs_features

[('cnn_pred', 'lstm_att_pred'),
 ('cnn_pred', 'lstm_pred'),
 ('cnn_pred', 'tf_pred'),
 ('lstm_att_pred', 'lstm_pred'),
 ('lstm_att_pred', 'tf_pred'),
 ('lstm_pred', 'tf_pred'),
 ('cnn_pred', 'lstm_att_pred', 'lstm_pred'),
 ('cnn_pred', 'lstm_att_pred', 'tf_pred'),
 ('cnn_pred', 'lstm_pred', 'tf_pred'),
 ('lstm_att_pred', 'lstm_pred', 'tf_pred'),
 ('cnn_pred', 'lstm_att_pred', 'lstm_pred', 'tf_pred')]

# Selected 5 Combination

In [9]:
# Bộ sai đầu tiên
# selected_combs = ['CNN*lstm_att_pred-tf_pred-Observed','CNN*cnn_pred-lstm_pred-Observed',
#                   'CNN*lstm_pred-tf_pred-Observed', 'CNN*cnn_pred-lstm_att_pred-lstm_pred-Observed',
#                   'LSTM*lstm_att_pred-lstm_pred-tf_pred-Observed'                 
#                  ]
# # Bộ này theo top 5 cố định RMSE
# selected_combs = ['LSTM*cnn_pred-tf_pred-Observed',
#                   'LSTM*lstm_att_pred-lstm_pred-Observed',     

#                   'LSTM*cnn_pred-lstm_att_pred-tf_pred-Observed',
#                   'CNN*lstm_att_pred-lstm_pred-Observed', 
#                   'LSTM*cnn_pred-lstm_pred-Observed'           
#                  ]

# Bộ này theo top 5 cố định MAE
# selected_combs = ['LSTM*lstm_att_pred-lstm_pred-tf_pred-Observed',
#                   'CNN*lstm_att_pred-tf_pred-Observed', 
#                   'LSTM*cnn_pred-lstm_att_pred-tf_pred-Observed',
#                   'CNN*lstm_att_pred-lstm_pred-Observed',
#                   'LSTM*cnn_pred-lstm_pred-Observed'
                
#                  ]
# selected_combs = ['LSTM*lstm_att_pred-lstm_pred-tf_pred-Observed',                    
#                   'CNN*lstm_att_pred-tf_pred-Observed'
                #  ]
selected_combs = ['LSTM*cnn_pred-lstm_att_pred-Observed',
                  'LSTM*lstm_att_pred-lstm_pred-Observed', 
                  'CNN*cnn_pred-lstm_pred-Observed',
                  'CNN*lstm_pred-tf_pred-Observed', 
                  'CNN*lstm_att_pred-lstm_pred-Observed',
                  'CNN*lstm_att_pred-lstm_pred-tf_pred-Observed',
                  'LSTM*cnn_pred-lstm_att_pred-lstm_pred-Observed',
                  'CNN*cnn_pred-lstm_att_pred-Observed'
                 ]

def is_in_Selected_combination(comb_string = ''):
    return any(comb_string in x for x in selected_combs)



# Support Functions and Train class 

In [10]:
import requests

def send_to_telegram(message):

    apiToken = '5908735099:AAGVSLrW62aXPBP-GrMvxoVgMsuJxXJpP1Q'
    chatID = '@ptn_announcement'
    apiURL = f'https://api.telegram.org/bot{apiToken}/sendMessage'

    try:
        response = requests.post(apiURL, json={'chat_id': chatID, 'text': message})
        print(response.text)
    except Exception as e:
        print(e)


def seed_everything(seed: int):
    import random, os    
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

seed_everything(42)

# Set hyperparameters as args using the Configuration class
class Configuration():
    def __init__(self):
      # lấy bộ test dài 36 tháng = 3 năm
        self.test_size = 36
        # là nhìn vào dữ liệu trước 3 tháng và dự phóng        
        self.look_back = 3
        # dự phóng 3 tháng
        self.n_predicted_month = 3
        self.n_features = 3
        self.seed = 42
        # mỗi phần tử x trong tập suppervise có độ lớn là 16 = 16 tháng
        self.batch_size = 16
        # self.device = torch.device("cuda")
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.epochs = 300
     

# Biến args lưu giá trị Global.
args = Configuration()


# Lưu ý sau khi stationnary thì dòng đầu tiên trừ cho dòng trước nữa là ko có cho nên về cơ bản sẽ thành NaN
# Cần remove dòng đầu tiên ra khỏi data để train và test. Cho nên cơ bản là data sau khi in ra sẽ thiếu 1 đơn vị đầu tiên
def convert_to_stationary(city_data):
    """Subtracts previous value for all cols except disease rates"""
    for col_name in city_data.columns:
        if col_name != 'Diarrhoea_rates':
            try:
                city_data[col_name] = city_data[col_name] - city_data[col_name].shift()
            except:
                print(col_name)
    return city_data

def impute_missing_value(city_data):
    """
    Cơ bản dữ liệu bị thiếu sót rất nhiều: Như Điện Biên 1997 -2003 là thiếu dữ liệu về bệnh
    Hàm này sẽ tự sinh ra dữ liệu bị thiếu. Nếu tháng nào không có số liệu thì tính như sau:
    12 tháng đầu không có số liệu thì gán = 0
    tháng 13-24 không có số liệu, sẽ lấy giá trị của tháng cùng kỳ năm trước
    tháng từ 24 trở đi sẽ lấy giá trị nhỏ nhất của 2 tháng cùng kỳ trong 2 năm gần nhất.
    Do Điện Biên bằng 0 nên sau khi xử lý từ 1997 -2003 là đều = 0.  
    """
    for col in city_data.columns:
        for index in range(len(city_data[col])):
            if np.isnan(city_data[col].iloc[index]):
                if index < 12:
                    city_data[col].iloc[index] = 0
                elif index >= 12 and index <= 24:
                    city_data[col].iloc[index] = city_data[col].iloc[index - 12]
                else:
                    city_data[col].iloc[index] = min(city_data[col].iloc[index - 12], city_data[col].iloc[index - 24])
    return city_data

def split_data(data, look_back ):
    """Splits data into train and test sets based on args (Configuration class)"""
    train = data[: -args.test_size]    
    test = data[-args.test_size - look_back: ]
    return train, test

def to_supervised(data, d_in=args.look_back, d_out=args.n_predicted_month, features_list=[]):
    """
    Frames time-series as supervised learning dataset.
    
    Args:
      d_in: lookback window
      d_out: number of predicted months
      features_list: list of all features **where last col is the disease incidence**

    Returns:
      Numpy arrays of disease incidence (y) and other predictors (X)
    """
    X, y = list(), list()
    for index, _ in enumerate(data):
        in_end = index + d_in
        out_end = in_end + d_out
        if out_end <= len(data):
            if len(features_list) == 0 :
                X.append(data[index: in_end, :])
            else:
                X.append(data[index: in_end, features_list])
            y.append(data[in_end: out_end, -1])
    return np.array(X), np.array(y)


def get_data(train_np, test_np, batch_size, list_selected_features, origin_city_meta_data):
    """
    Returns important feature list and data formatted for input into Pytorch 
    models
    """ 
    # origin_city_meta_data chứa data mẫu để lấy index 
    # important_features = arrary index nên ta phải truyền indexex của columns
    important_features = [origin_city_meta_data.columns.get_loc(c) for c in list_selected_features if c in origin_city_meta_data]  

    train_X, train_y = to_supervised(train_np, features_list=important_features)
    test_X, test_y = to_supervised(test_np, features_list=important_features)
    train_tensor = TensorDataset(torch.from_numpy(train_X), torch.from_numpy(train_y))
    test_tensor = (torch.from_numpy(test_X), torch.from_numpy(test_y))

    train_loader = DataLoader(train_tensor, batch_size=batch_size, shuffle=True)

    return important_features, train_loader, test_tensor

#Define Pytorch LSTM model
class MultiVariateLSTM(nn.Module):
    def __init__(self, n_feature=3, n_layers=2, hidden_size=50):
        super(MultiVariateLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=n_feature, hidden_size=hidden_size, num_layers=n_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, args.n_predicted_month)
        self.loss_fn = nn.MSELoss()
        self.sigma = nn.Parameter(torch.ones(args.n_predicted_month))
    
    def forward(self, X_batch, y_batch=None):
        output, (last_hidden, _) = self.lstm(X_batch)
        last_hidden_vector = output[:, -1, :]
        y_predicted = self.linear(last_hidden_vector)
        if y_batch != None:
            assert y_predicted.size() == y_batch.size()
            loss = self.loss_fn(y_predicted, y_batch)
            loss = 0.5 * loss / self.sigma**2
            loss = loss.sum() + torch.log(1 + self.sigma.prod())
            return y_predicted, loss
            #return y_predicted, self.loss_fn(y_predicted, y_batch)
        else:
            return y_predicted
    
    def predict(self, X):
        X = torch.tensor(X, device=args.device)
        return self.forward(X)

#Define Pytorch LSTM-ATT model
class MultiVariateLSTM_Attention(nn.Module):
    def __init__(self, n_feature=3, n_layers=2, hidden_size=50):
        super(MultiVariateLSTM_Attention, self).__init__()
        self.lstm = nn.LSTM(input_size=n_feature, hidden_size=hidden_size, num_layers=n_layers, batch_first=True)
        self.attention_linear = nn.Linear(hidden_size, hidden_size)
        # self.linear = nn.Linear(hidden_size*2, args.n_predicted_month)
        self.linear = nn.Linear(hidden_size, args.n_predicted_month)
        self.loss_fn = nn.MSELoss()
        self.sigma = nn.Parameter(torch.ones(args.n_predicted_month))
    
    def forward(self, X_batch, y_batch=None):
        output, (last_hidden, _) = self.lstm(X_batch)
        last_hidden_vector = last_hidden[-1]
        remain_hidden_vector = output
        e_t = remain_hidden_vector.bmm(self.attention_linear(last_hidden_vector).unsqueeze(2)).squeeze(-1)
        alpha_t = F.softmax(e_t, dim=1)
        attenion_vector = remain_hidden_vector.transpose(2, 1).bmm(alpha_t.unsqueeze(2)).squeeze(-1)
        # combine_vector = torch.cat((last_hidden_vector, attenion_vector), dim=1)
        # combine_vector = last_hidden_vector + attenion_vector
        y_predicted = self.linear(attenion_vector)
        if y_batch != None:
            assert y_predicted.size() == y_batch.size()
            loss = self.loss_fn(y_predicted, y_batch)
            loss = 0.5 * loss / self.sigma**2
            loss = loss.sum() + torch.log(1 + self.sigma.prod())
            return y_predicted, loss
            # return y_predicted, self.loss_fn(y_predicted, y_batch)
        else:
            return y_predicted
    
    def predict(self, X):
        X = torch.tensor(X, device=args.device)
        return self.forward(X)

# Define Pytorch CNN model
class MultivariateCNN(nn.Module):
    def __init__(self, num_filters=[100, 100, 100], dropout=0.01):
        super(MultivariateCNN, self).__init__()
        self.loss_fn = loss = nn.MSELoss()
        self.filter_sizes = [1, 2, 3]
        self.conv1d_list = nn.ModuleList([nn.Conv1d(args.n_features, num_filters[i], self.filter_sizes[i]) for i in range(len(self.filter_sizes))])
        self.linear = nn.Linear(np.sum(num_filters), args.n_predicted_month)
        self.dropout = nn.Dropout(dropout)
        self.sigma = nn.Parameter(torch.ones(args.n_predicted_month))

    def forward(self, X_batch, y_batch=None):
        X_batch = X_batch.permute(0, 2, 1)  #(batch_size, n_features, n_look_back)
        X_conv_list = [F.relu(conv1d(X_batch)) for conv1d in self.conv1d_list]
        X_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2]) for x_conv in X_conv_list]
        X_fc = torch.cat([x_pool.squeeze(dim=2) for x_pool in X_pool_list], dim=1)
        y_predicted = self.linear(self.dropout(X_fc))
        if y_batch != None:
            assert y_predicted.size() == y_batch.size()
            loss = self.loss_fn(y_predicted, y_batch)
            loss = 0.5 * loss / self.sigma**2
            loss = loss.sum() + torch.log(1 + self.sigma.prod())
            return y_predicted, loss
            # return y_predicted, self.loss_fn(y_predicted, y_batch)
        else:
            return y_predicted

    def predict(self, X):
        X = torch.tensor(X, device=args.device)
        return self.forward(X)

# Transformer Model
# Transformer Model
class PositionalEncoder(nn.Module):
    def __init__(self, n_feature=3, dropout=0.1, max_length=24):
        super().__init__()
        
        self.hidden_dim = n_feature
        self.max_length = max_length
        self.dropout = nn.Dropout(dropout)
        
        pe = torch.zeros(max_length, self.hidden_dim)
        for pos in range(max_length):
            for i in range(0, self.hidden_dim, 2):
                pe[pos, i] = math.sin(pos/(10000**(2*i/self.hidden_dim)))
                if i + 1 < self.hidden_dim:
                    pe[pos, i+1] = math.cos(pos/(10000**((2*i+1)/self.hidden_dim)))
        pe = pe.unsqueeze(0)        
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        x = x*math.sqrt(self.hidden_dim)
        length = x.size(1)
        pe = Variable(self.pe[:, :length], requires_grad=False)
        if x.is_cuda:
            pe.cuda()
        x = x + pe
        x = self.dropout(x)
        
        return x

class TransformerModel(nn.Module):
    def __init__(self, d_input, n_look_back, hidden_size, n_layers, dropout=0.5):
        super(TransformerModel, self).__init__()
        self.hidden_size = hidden_size

        self.pe = PositionalEncoder(n_feature=args.n_features, dropout=dropout)

        encoder_layers = nn.TransformerEncoderLayer(d_model=d_input, nhead=d_input, dim_feedforward=hidden_size, dropout=dropout, activation='gelu')
        # encoder_layers = nn.TransformerEncoderLayer(d_model=d_input, nhead=d_input, dropout=dropout, activation='gelu')
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, n_layers)
        self.decoder = nn.Linear(d_input*n_look_back, args.n_predicted_month)

        self.sigma = nn.Parameter(torch.ones(args.n_predicted_month))
        
        self.loss_fn = nn.MSELoss()
        
    
    def forward(self, X_batch, y_batch=None):
        X_batch = self.pe(X_batch)
        X_batch = self.transformer_encoder(X_batch)
        X_batch = X_batch.view(X_batch.size(0), -1)
        
        y_predicted = self.decoder(X_batch)
        if y_batch != None:
            assert y_predicted.size() == y_batch.size()
            loss = self.loss_fn(y_predicted, y_batch)
            loss = 0.5 * loss / self.sigma**2
            loss = loss.sum() + torch.log(1 + self.sigma.prod())
            return y_predicted, loss
        else:
            return y_predicted
        return X_batch
    
    def predict(self, X):
        with torch.no_grad():
            X = torch.tensor(X, device=args.device)
        return self.forward(X).squeeze()

# Create class to train and evaluate models
class Trainer():
    def __init__(self, city, model_type, learning_rate, important_features, train_loader, test_tensor, n_layers=2, hidden_size=128, num_filters=[100, 100, 100], dropout=0.01):
        """
        Initialise trainer, allowing input of LSTM, LSTM-ATT, or CNN 
        hyperparameters. Adam optimiser used for all models.
        """
        self.model_type = model_type    
        self.city = city    
        self.model = self.init_model(model_type, n_layers, hidden_size, num_filters, dropout, city)
        self.model.double().to(args.device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=learning_rate)
        self.important_features, self.train_loader, self.test_tensor = important_features, train_loader, test_tensor
    
    def init_model(self, model_type, n_layers, hidden_size, num_filters, dropout, city):        
        #Get hyper param by name and set for n_layers, hidden_size, num_filters, dropout and overwrite the input param  
        # n_layers, hidden_size,num_filters, dropout= get_opt_param_by_name( algo = model_type, city = city)
    
        """Initialise a model based on whether LSTM, LSTM-ATT, or CNN is chosen."""
        if model_type.lower() == 'lstm':
            model = MultiVariateLSTM(args.n_features, n_layers, hidden_size)
        elif model_type.lower() == 'lstm_attention':
            model = MultiVariateLSTM_Attention(args.n_features, n_layers, hidden_size)
        elif model_type.lower() == 'cnn':
            model = MultivariateCNN(num_filters, dropout)
        elif model_type.lower() == 'transformers':
            model = TransformerModel(d_input=args.n_features, n_look_back=args.look_back, hidden_size=hidden_size, n_layers=n_layers, dropout=dropout)
        return model

    def step(self, batch):
        self.model.train()
        X_batch, y_batch = tuple(t.to(args.device) for t in batch)
        self.optimizer.zero_grad()
        y_pred, loss = self.model.forward(X_batch, y_batch)
        loss.backward()
        self.optimizer.step()
        return loss.mean().item()

    def validation(self):
        self.model.eval()
        eval_loss = 0.0

        result = {}

        y_true = np.array([])
        y_pred = np.array([])

        X_batch, y_batch = tuple(t.to(args.device) for t in self.test_tensor)
        with torch.no_grad():
            outputs, loss = self.model.forward(X_batch, y_batch)
            eval_loss = loss.mean().item()

        return eval_loss

    def train(self, epochs=20):
        # print('param of model')
        # print('hidensize',self.model.hidden_size)
        # print('nlayer',self.model.n_layers)  
        best_lost = float("inf")
        best_model = None
        for epoch in range(epochs):
            total_loss = 0.0
            for batch in self.train_loader:
                loss = self.step(batch)
                total_loss += loss
            train_loss = total_loss/len(self.train_loader)
            eval_loss = self.validation()
            if eval_loss < best_lost:
                best_lost = eval_loss
                best_model = copy.deepcopy(self.model)
            if (epoch + 1) == epochs or (epoch + 1) in [c + 1 for c in range(epochs) if c % int(epochs/4) == 0]:
                print(f"Epoch: {epoch:2}/{epochs:2} - train_loss: {train_loss:.4f} - test_loss: {eval_loss:4f}")
        self.model = best_model
        self.model.eval()
        return None



    # Lưu model vào trong thư mục models
    def save_model_to(self, path = '', city =''):       
        torch.save(self.model, path)

    def load_model_to(self, path = ''):       
        return torch.load(path)

    def evaluate_model(self, np_data=None, plot=True, scaled=True, city=None, k_steps=None, y_scaler =None):
        assert scaled, "data must be scaled"
        self.model.eval()
        tensor_data = torch.from_numpy(np_data)
        rmse_list = []
        mae_list = [] 
        mape_list = []

        y_predicted_list = []
        y_true_list = []

        for k_steps in range(1, args.n_predicted_month + 1):
            y_predicted = []
            y_true = []
            for index in range(tensor_data.size(0) - args.look_back):
                X = tensor_data[index: index + args.look_back, self.important_features]
                # yhat = self.model.predict(X.unsqueeze(0)).squeeze()

                yhat = self.model.predict(X.unsqueeze(0))
                yhat = yhat.squeeze()

                y_predicted.append(yhat.detach().cpu().numpy()[k_steps - 1])
                y_true.append(tensor_data[index + args.look_back, -1].detach().cpu().numpy())

            y_predicted = y_scaler.inverse_transform(np.array(y_predicted).reshape(-1, 1)).reshape(-1, )
            y_true = y_scaler.inverse_transform(np.array(y_true).reshape(-1, 1)).reshape(-1, )

            

            rmse = mean_squared_error(y_true, y_predicted, squared=False)
            mae = mean_absolute_error(y_true, y_predicted)
            mape = mean_absolute_percentage_error(y_true, y_predicted)

            rmse_list.append(rmse)
            mae_list.append(mae)
            mape_list.append(mape)

            print('City: '+self.city+'  _algo:'+self.model_type+'  -RMSE: '+str(rmse))          
            if plot==True:
              plt.grid(True)
              plt.plot(y_predicted, label='predicted')
              plt.plot(y_true, label='actual')
              plt.title(f"k-steps = {k_steps} - city: "+self.city+'  _algo:'+self.model_type+'  -RMSE: '+str(rmse))
              plt.legend()
              plt.show()

              plt.show()
            y_predicted_list.append(y_predicted)
            y_true_list.append(y_true)

        return y_true_list, y_predicted_list, rmse_list, mae_list, mape_list

# Get optimized Hyper param

In [11]:
def convert_str_num_filter_array(str_filter = ''):  
  arr_filter = str_filter.split(',')  
  int_arr_filter= [int(x) for x in arr_filter]
  return int_arr_filter


opt_param_cnn = pd.read_excel(prj_path_opt+'diarrhoea_opt_hyperparam_cnn.xlsx', index_col = 'City')
opt_param_lstm = pd.read_excel(prj_path_opt+'diarrhoea_opt_hyperparam_lstm.xlsx', index_col = 'City')
opt_param_lstm_att = pd.read_excel(prj_path_opt+'diarrhoea_opt_hyperparam_lstm_att.xlsx', index_col = 'City')
opt_param_transformer = pd.read_excel(prj_path_opt+'diarrhoea_opt_hyperparam_transformer.xlsx', index_col = 'City')

def get_opt_param_by_name(algo = 'transformers', city = None):
  if algo == 'cnn':
    city_param = opt_param_cnn.loc[city]
    #load  param
    n_layers = ''
    hidden_size = ''
    num_filters = convert_str_num_filter_array(city_param['Num. filters'])
    dropout = float(city_param['Dropout rate'])
    learning_rate = float(city_param['Learning rate'])
    epoch = int(city_param['Epochs'])
    lookback_window = int(city_param['Lookback Window'])
  elif algo == 'lstm':
    city_param = opt_param_lstm.loc[city]
    #load  param
    n_layers = int(city_param['n Layers'])
    hidden_size = int(city_param['Hiden Size'])
    num_filters = ''
    dropout = ''
    learning_rate = float(city_param['Learning rate'])
    epoch = int(city_param['Epochs'])
    lookback_window = int(city_param['Lookback Window'])
  elif algo == 'lstm_attention':
    city_param = opt_param_lstm_att.loc[city]
    #load  param
    n_layers = int(city_param['n Layers'])
    hidden_size = int(city_param['Hiden Size'])
    num_filters = ''
    dropout = ''
    learning_rate = float(city_param['Learning rate'])
    epoch = int(city_param['Epochs'])
    lookback_window = int(city_param['Lookback Window'])
  elif algo == 'transformers':
    city_param = opt_param_transformer.loc[city]
    #load  param
    n_layers = int(city_param['n Layers'])
    hidden_size = int(city_param['Hidden Size'])
    num_filters = ''
    dropout = float(city_param['Dropout rate'])
    learning_rate = float(city_param['Learning rate'])
    epoch = int(city_param['Epochs'])
    lookback_window = int(city_param['Lookback Window'])
  return n_layers, hidden_size, num_filters, dropout, learning_rate, epoch, lookback_window


# Manual Check param before train
n_layers, hidden_size, num_filters, dropout, learning_rate, epoch, lookback_window= get_opt_param_by_name(algo='lstm', city='Cao Bằng')
lookback_window,epoch, hidden_size, n_layers,learning_rate, num_filters, dropout

(3, 190, 168, 6, 0.004015679480821153, '', '')

# Create Cross validation Train/Test set

In [12]:
# Cross validation chỉ trên chiều dài của bộ train = 212 (1997 - 1/1/2018 bỏ đi (36 + 3 windows + 1 station))
# tạo l_train_indices\l_test_indices
data = list(range(212))
min_train_size = 72 # Chiều dài ban đầu train size = 6 năm = 72
test_size = 36 # chiều dài test size 36 tháng
l_train_indices= []
l_test_indices = []
for i in range(min_train_size, len(data)-test_size+1):
    train = np.array(data[0:i])
    test = np.array(data[i:i+test_size])    
    l_train_indices.append(train)
    l_test_indices.append(test)

len(l_test_indices)

105

# Function Tạo và Predict. Important (epoch = 1 for test)

In [13]:
def do_train_predict(city ='', algo = '', list_selected_features = '', origin_city_meta_test_set= '', y_scaler = '', s_selected_features= '', train ='', test ='', sample_idx ='', combination_idx= ''):
    
    metric_reslt = pd.DataFrame()
    n_layers, hidden_size, num_filters, dropout, learning_rate, epoch, lookback_window = get_opt_param_by_name( algo = algo, city = city)
    # Get data to run model
    important_features, train_loader, test_tensor = get_data(train, test, args.batch_size, list_selected_features, origin_city_meta_test_set)   

#     epoch = 1 # comment when do experiment
    
    if( algo =='cnn'): 
        algo = 'CNN'    # upper case only
        trainer = Trainer(
                            model_type='cnn',  
                            city = city,                    
                            learning_rate=learning_rate,
                            important_features=important_features,
                            train_loader=train_loader,
                            test_tensor=test_tensor,
                            num_filters= num_filters, 
                            dropout=dropout )

    if( algo =='lstm'):
        algo = 'LSTM'    # upper case only        
        trainer = Trainer(
                            model_type='lstm', 
                            city = city,
                            learning_rate=learning_rate,
                            important_features=important_features,
                            train_loader=train_loader,
                            test_tensor=test_tensor,
                            n_layers=n_layers,
                            hidden_size=hidden_size)

    if( algo =='lstm_attention'): 
        algo = 'LSTM ATT'    # upper case only       
        trainer = Trainer(
                            model_type= 'lstm_attention',
                            city = city,
                            learning_rate= learning_rate,
                            important_features=important_features,
                            train_loader=train_loader,
                            test_tensor=test_tensor,
                            n_layers= n_layers,
                            hidden_size=hidden_size )

    if( algo =='transformers'): 
        algo = 'TF'    # upper case only       
        trainer = Trainer(
                            model_type='transformers',
                            city = city, 
                            learning_rate=learning_rate,
                            important_features=important_features,
                            train_loader=train_loader,
                            test_tensor=test_tensor,
                            n_layers=n_layers,
                            hidden_size=hidden_size,
                            dropout=dropout)

    
    # Train model with province-specific epochs
    trainer.train(epochs=epoch)  
    # Evaluate test set   
    y_true_test, y_pred_test, rmse_list, mae_list, mape_list = trainer.evaluate_model(np_data=test, plot= False, scaled=True, city=city, y_scaler= y_scaler)
        
    metric_reslt = pd.DataFrame({
                                'City': city,
                                'Meta_Model_Name': algo, 
                                'sample_index':sample_idx,
                                'Based_Models': s_selected_features,                                                      
                                # algo+'_'+str(combination_idx)+'_rmse': rmse_list[0],
                                # algo+'_'+str(combination_idx)+'_mae': mae_list[0]
                                algo+'*'+s_selected_features+'_rmse': rmse_list[0],
                                algo+'*'+s_selected_features+'_mae': mae_list[0]
                                }, index=[0]) # do có 1 dòng nên pandas cần index = 0    
  

        
    return metric_reslt

# Main Cross Validation Metrics Creation
Tạo ra 105 bộ validation. Xuất ra Metrics (RMSE và MAE) cho 5 bộ giải thuật chọn lựa. (10 cột cho 1 dòng validation )

In [ ]:
dt_started = datetime.now()

l_errCity = {} # có lỗi sẽ lưu vào đây, kiểm tra ngay cell sau


# Với mỗi thành phố, loop qua từng bộ tr/validation, cho train/predict N tổ hợp models
for city in cities:
  pd_all_rows_new_meta_data = pd.DataFrame()

  for sample_index in  range(0,len(l_train_indices)):

    pd_one_row_new_meta_data = pd.DataFrame()
    combination_idx = 0
    for combination_features in combs_features:            
      list_selected_features = [x for x in combination_features] +['Observed'] # Lấy danh sách tên các features + cột Observed 
      s_selected_features = '-'.join([str(elem) for elem in list_selected_features])      

      # Có trong danh sách chọn lựa thì thực hiện dự đoán
      if((is_in_Selected_combination('CNN*'+s_selected_features)==True) or (is_in_Selected_combination('LSTM*'+s_selected_features) == True)):        
        args.n_features = len(list_selected_features) 
        args.look_back = meta_lookback_window  
        cnn_metrics = pd.DataFrame()
        lstm_metrics = pd.DataFrame()

        try:
          # Lấy data từ train set và chia ra theo valiation
          origin_city_meta_train_set = meta_train_set_city[city]
          origin_city_meta_test_set = meta_test_set_city[city]
          
          train = origin_city_meta_train_set.iloc[l_train_indices[sample_index]]
          test = origin_city_meta_train_set.iloc[l_test_indices[sample_index]]        

          # Data train/test processing for all Meta model
          train = train.astype(float)
          test = test.astype(float)

          train = train.replace((np.inf, -np.inf, np.nan), 0)
          test = test.replace((np.inf, -np.inf, np.nan), 0) 
        
          # Đoạn này impute các trường hợp rỗng
          train = impute_missing_value(train)
          test = impute_missing_value(test)

          # Fit data scaler to training data
          full_scaler = MinMaxScaler().fit(train)
          y_scaler = MinMaxScaler().fit(train.values[:, -1].reshape(-1, 1))

          # Scale train and test data
          train = full_scaler.transform(train)
          test = full_scaler.transform(test)

          if(is_in_Selected_combination('CNN*'+s_selected_features)==True):
            ########################################
            ########## CNN 
            ########################################
            cnn_metrics = do_train_predict(city =city, algo='cnn', list_selected_features = list_selected_features,
                                                origin_city_meta_test_set= origin_city_meta_test_set, y_scaler = y_scaler, 
                                                s_selected_features= s_selected_features, train = train, test = test, 
                                                sample_idx = sample_index,combination_idx= combination_idx)

          if(is_in_Selected_combination('LSTM*'+s_selected_features)==True):
            ########################################
            ########## LSTM 
            ########################################
            lstm_metrics = do_train_predict(city =city, algo='lstm', list_selected_features = list_selected_features,
                                                origin_city_meta_test_set= origin_city_meta_test_set, y_scaler = y_scaler, 
                                                s_selected_features= s_selected_features, train = train, test = test, 
                                                sample_idx = sample_index,combination_idx= combination_idx)
          

          
          
          ###############################   
          ## Khúc này rất thủ công
          if(cnn_metrics.empty==False and lstm_metrics.empty ==False):              
            join_2_results = pd.concat([cnn_metrics.iloc[:, [4, 5]], lstm_metrics.iloc[:, [4, 5]]], axis=1)
          
          if(cnn_metrics.empty== False and lstm_metrics.empty ==True):
            join_2_results = cnn_metrics.iloc[:, [4, 5]]
          
          if(cnn_metrics.empty== True and lstm_metrics.empty ==False):
            join_2_results = lstm_metrics.iloc[:, [4, 5]]

          if(pd_one_row_new_meta_data.empty):
            pd_one_row_new_meta_data = join_2_results
          else:
            pd_one_row_new_meta_data = pd.concat([join_2_results, pd_one_row_new_meta_data], axis=1)       

          combination_idx = combination_idx + 1
        except Exception as e:
          l_errCity[city] = e
          break    

        
    # Hết vòng lập pd_one_row_new_meta_data chứa 5 bộ kết quả của 1 sameple    
    # Qua sample kế tiếp   
    pd_all_rows_new_meta_data = pd.concat([pd_all_rows_new_meta_data, pd_one_row_new_meta_data], axis=0) 

  
  # Kết thúc hết tất cả validation -> Lưu kết quả cuối cùng ứng với từng thành phố
  pd_all_rows_new_meta_data.to_excel(com_path_validation_metrics+ city+'_5_comb_cv_metrics.xlsx')
  
  # Telegram vào
  send_to_telegram('Server Chạy xong city: '+ city)

dt_ended = datetime.now()
run_time = round((dt_ended - dt_started).total_seconds()/60)
print('kết thúc train trong:', run_time)
print ('danh sách error city: ')
print(l_errCity)


Epoch:  0/450 - train_loss: 1.1668 - test_loss: 1.061506
Epoch: 112/450 - train_loss: 0.6733 - test_loss: 0.648010
Epoch: 224/450 - train_loss: 0.5714 - test_loss: 0.564880
Epoch: 336/450 - train_loss: 0.5110 - test_loss: 0.492626
Epoch: 448/450 - train_loss: 0.4568 - test_loss: 0.431825
Epoch: 449/450 - train_loss: 0.4411 - test_loss: 0.431513
City: Kon Tum  _algo:cnn  -RMSE: 121.88251675565323
City: Kon Tum  _algo:cnn  -RMSE: 119.01361130521751
City: Kon Tum  _algo:cnn  -RMSE: 121.98464661503563
Epoch:  0/260 - train_loss: 1.2899 - test_loss: 1.272298
Epoch: 65/260 - train_loss: 0.6698 - test_loss: 0.638085
Epoch: 130/260 - train_loss: 0.5857 - test_loss: 0.554340
Epoch: 195/260 - train_loss: 0.5297 - test_loss: 0.486883
Epoch: 259/260 - train_loss: 0.4557 - test_loss: 0.431662
City: Kon Tum  _algo:lstm  -RMSE: 124.83069554069957
City: Kon Tum  _algo:lstm  -RMSE: 121.27106610196647
City: Kon Tum  _algo:lstm  -RMSE: 121.01930174656344
Epoch:  0/450 - train_loss: 0.9926 - test_loss: 0.

Epoch:  0/260 - train_loss: 1.3512 - test_loss: 1.269056
Epoch: 65/260 - train_loss: 0.6561 - test_loss: 0.621907
Epoch: 130/260 - train_loss: 0.5805 - test_loss: 0.538916
Epoch: 195/260 - train_loss: 0.5002 - test_loss: 0.470225
Epoch: 259/260 - train_loss: 0.4621 - test_loss: 0.413781
City: Kon Tum  _algo:lstm  -RMSE: 127.93487073693353
City: Kon Tum  _algo:lstm  -RMSE: 121.31541208840729
City: Kon Tum  _algo:lstm  -RMSE: 120.34913644745966
Epoch:  0/450 - train_loss: 1.2221 - test_loss: 1.153055
Epoch: 112/450 - train_loss: 0.6733 - test_loss: 0.636341
Epoch: 224/450 - train_loss: 0.5750 - test_loss: 0.551145
Epoch: 336/450 - train_loss: 0.5036 - test_loss: 0.480892
Epoch: 448/450 - train_loss: 0.4262 - test_loss: 0.422451
Epoch: 449/450 - train_loss: 0.4216 - test_loss: 0.421938
City: Kon Tum  _algo:cnn  -RMSE: 118.15040261929954
City: Kon Tum  _algo:cnn  -RMSE: 112.73977414771869
City: Kon Tum  _algo:cnn  -RMSE: 114.72714397654039
Epoch:  0/450 - train_loss: 1.3761 - test_loss: 1.

Epoch: 112/450 - train_loss: 0.7004 - test_loss: 0.636328
Epoch: 224/450 - train_loss: 0.5909 - test_loss: 0.553302
Epoch: 336/450 - train_loss: 0.4982 - test_loss: 0.482892
Epoch: 448/450 - train_loss: 0.4389 - test_loss: 0.423804
Epoch: 449/450 - train_loss: 0.4365 - test_loss: 0.423627
City: Kon Tum  _algo:cnn  -RMSE: 115.4677276694422
City: Kon Tum  _algo:cnn  -RMSE: 115.19918217620398
City: Kon Tum  _algo:cnn  -RMSE: 117.03943347167942
Epoch:  0/450 - train_loss: 1.0398 - test_loss: 0.948702
Epoch: 112/450 - train_loss: 0.6696 - test_loss: 0.638218
Epoch: 224/450 - train_loss: 0.5772 - test_loss: 0.554948
Epoch: 336/450 - train_loss: 0.4929 - test_loss: 0.483555
Epoch: 448/450 - train_loss: 0.4289 - test_loss: 0.423961
Epoch: 449/450 - train_loss: 0.4357 - test_loss: 0.423330
City: Kon Tum  _algo:cnn  -RMSE: 118.50288249482628
City: Kon Tum  _algo:cnn  -RMSE: 118.95620744456728
City: Kon Tum  _algo:cnn  -RMSE: 118.2405942704605
Epoch:  0/260 - train_loss: 1.2609 - test_loss: 1.147

Epoch: 112/450 - train_loss: 0.6717 - test_loss: 0.635549
Epoch: 224/450 - train_loss: 0.5727 - test_loss: 0.552608
Epoch: 336/450 - train_loss: 0.4879 - test_loss: 0.481069
Epoch: 448/450 - train_loss: 0.4224 - test_loss: 0.421898
Epoch: 449/450 - train_loss: 0.4242 - test_loss: 0.421486
City: Kon Tum  _algo:cnn  -RMSE: 117.25201541069164
City: Kon Tum  _algo:cnn  -RMSE: 118.23470060709577
City: Kon Tum  _algo:cnn  -RMSE: 118.60865652905683
Epoch:  0/260 - train_loss: 1.1742 - test_loss: 1.060701
Epoch: 65/260 - train_loss: 0.6762 - test_loss: 0.631528
Epoch: 130/260 - train_loss: 0.5838 - test_loss: 0.544257
Epoch: 195/260 - train_loss: 0.5118 - test_loss: 0.472488
Epoch: 259/260 - train_loss: 0.4540 - test_loss: 0.415462
City: Kon Tum  _algo:lstm  -RMSE: 123.72375115378037
City: Kon Tum  _algo:lstm  -RMSE: 119.62000533589847
City: Kon Tum  _algo:lstm  -RMSE: 118.44170213198841
Epoch:  0/450 - train_loss: 1.4823 - test_loss: 1.231714
Epoch: 112/450 - train_loss: 0.6882 - test_loss: 0

Epoch: 65/260 - train_loss: 0.6828 - test_loss: 0.640273
Epoch: 130/260 - train_loss: 0.5880 - test_loss: 0.551239
Epoch: 195/260 - train_loss: 0.5193 - test_loss: 0.477382
Epoch: 259/260 - train_loss: 0.4541 - test_loss: 0.423054
City: Kon Tum  _algo:lstm  -RMSE: 120.98679230017906
City: Kon Tum  _algo:lstm  -RMSE: 118.26425676035291
City: Kon Tum  _algo:lstm  -RMSE: 119.3498032924865
Epoch:  0/450 - train_loss: 1.1777 - test_loss: 1.032324
Epoch: 112/450 - train_loss: 0.6774 - test_loss: 0.644865
Epoch: 224/450 - train_loss: 0.5681 - test_loss: 0.560020
Epoch: 336/450 - train_loss: 0.4928 - test_loss: 0.488903
Epoch: 448/450 - train_loss: 0.4027 - test_loss: 0.431526
Epoch: 449/450 - train_loss: 0.4172 - test_loss: 0.430188
City: Kon Tum  _algo:cnn  -RMSE: 123.4077292136405
City: Kon Tum  _algo:cnn  -RMSE: 123.09510488121383
City: Kon Tum  _algo:cnn  -RMSE: 126.64616863720643
Epoch:  0/260 - train_loss: 1.2939 - test_loss: 1.138652
Epoch: 65/260 - train_loss: 0.6628 - test_loss: 0.62

...

Epoch: 95/380 - train_loss: 0.1696 - test_loss: 0.101533
Epoch: 190/380 - train_loss: 0.1547 - test_loss: 0.098599
Epoch: 285/380 - train_loss: 0.1641 - test_loss: 0.108181
Epoch: 379/380 - train_loss: 0.1563 - test_loss: 0.111614
City: Long An  _algo:cnn  -RMSE: 14.334994366424597
City: Long An  _algo:cnn  -RMSE: 16.194060996358687
City: Long An  _algo:cnn  -RMSE: 16.822114920025545
Epoch:  0/500 - train_loss: 0.8455 - test_loss: 0.732101
Epoch: 125/500 - train_loss: 0.2158 - test_loss: 0.166695
Epoch: 250/500 - train_loss: 0.1841 - test_loss: 0.123378
Epoch: 375/500 - train_loss: 0.1771 - test_loss: 0.105416
Epoch: 499/500 - train_loss: 0.1738 - test_loss: 0.113184
City: Long An  _algo:lstm  -RMSE: 16.03466597335973
City: Long An  _algo:lstm  -RMSE: 16.010836217918307
City: Long An  _algo:lstm  -RMSE: 16.67903505239229
Epoch:  0/380 - train_loss: 0.8066 - test_loss: 0.691051
Epoch: 95/380 - train_loss: 0.2492 - test_loss: 0.096334
Epoch: 190/380 - train_loss: 0.1641 - test_loss: 0.09

Epoch:  0/500 - train_loss: 0.8114 - test_loss: 0.703288
Epoch: 125/500 - train_loss: 0.2077 - test_loss: 0.158035
Epoch: 250/500 - train_loss: 0.1916 - test_loss: 0.100121
Epoch: 375/500 - train_loss: 0.1740 - test_loss: 0.117934
Epoch: 499/500 - train_loss: 0.1730 - test_loss: 0.112069
City: Long An  _algo:lstm  -RMSE: 19.327058664867167
City: Long An  _algo:lstm  -RMSE: 17.539135864083644
City: Long An  _algo:lstm  -RMSE: 17.243631341668955
Epoch:  0/380 - train_loss: 0.9001 - test_loss: 0.676982
Epoch: 95/380 - train_loss: 0.1618 - test_loss: 0.106933
Epoch: 190/380 - train_loss: 0.1653 - test_loss: 0.094133
Epoch: 285/380 - train_loss: 0.1579 - test_loss: 0.102287
Epoch: 379/380 - train_loss: 0.1394 - test_loss: 0.101155
City: Long An  _algo:cnn  -RMSE: 17.160096910690193
City: Long An  _algo:cnn  -RMSE: 18.016205562669654
City: Long An  _algo:cnn  -RMSE: 17.5342152763767
Epoch:  0/380 - train_loss: 0.8670 - test_loss: 0.694124
Epoch: 95/380 - train_loss: 0.1743 - test_loss: 0.111

Epoch: 379/380 - train_loss: 0.1600 - test_loss: 0.086176
City: Long An  _algo:cnn  -RMSE: 15.073403368683476
City: Long An  _algo:cnn  -RMSE: 14.7496825522094
City: Long An  _algo:cnn  -RMSE: 15.190948500563685
Epoch:  0/500 - train_loss: 0.9827 - test_loss: 0.774744
Epoch: 125/500 - train_loss: 0.2110 - test_loss: 0.150083
Epoch: 250/500 - train_loss: 0.1817 - test_loss: 0.091722
Epoch: 375/500 - train_loss: 0.1736 - test_loss: 0.087848
Epoch: 499/500 - train_loss: 0.1677 - test_loss: 0.087096
City: Long An  _algo:lstm  -RMSE: 13.958428807038862
City: Long An  _algo:lstm  -RMSE: 12.274667097241988
City: Long An  _algo:lstm  -RMSE: 11.922085512961539
Epoch:  0/380 - train_loss: 0.8291 - test_loss: 0.679409
Epoch: 95/380 - train_loss: 0.1673 - test_loss: 0.082200
Epoch: 190/380 - train_loss: 0.1528 - test_loss: 0.085524
Epoch: 285/380 - train_loss: 0.1532 - test_loss: 0.081032
Epoch: 379/380 - train_loss: 0.1467 - test_loss: 0.074476
City: Long An  _algo:cnn  -RMSE: 12.097685919475577


Epoch:  0/380 - train_loss: 0.8812 - test_loss: 0.686545
Epoch: 95/380 - train_loss: 0.1655 - test_loss: 0.110076
Epoch: 190/380 - train_loss: 0.1572 - test_loss: 0.103982
Epoch: 285/380 - train_loss: 0.1557 - test_loss: 0.105027
Epoch: 379/380 - train_loss: 0.1474 - test_loss: 0.091631
City: Long An  _algo:cnn  -RMSE: 14.186570932142741
City: Long An  _algo:cnn  -RMSE: 13.777625355173349
City: Long An  _algo:cnn  -RMSE: 15.342384860300358
Epoch:  0/500 - train_loss: 0.8446 - test_loss: 0.707614
Epoch: 125/500 - train_loss: 0.2103 - test_loss: 0.145429
Epoch: 250/500 - train_loss: 0.1717 - test_loss: 0.092064
Epoch: 375/500 - train_loss: 0.1706 - test_loss: 0.089812
Epoch: 499/500 - train_loss: 0.1680 - test_loss: 0.097993
City: Long An  _algo:lstm  -RMSE: 12.974671535738455
City: Long An  _algo:lstm  -RMSE: 13.544890961476085
City: Long An  _algo:lstm  -RMSE: 12.235104539183661
Epoch:  0/380 - train_loss: 0.8791 - test_loss: 0.687458
Epoch: 95/380 - train_loss: 0.1698 - test_loss: 0.1

Epoch: 379/380 - train_loss: 0.1458 - test_loss: 0.108335
City: Long An  _algo:cnn  -RMSE: 14.801893477596249
City: Long An  _algo:cnn  -RMSE: 12.780287665216488
City: Long An  _algo:cnn  -RMSE: 19.140827933443298
Epoch:  0/500 - train_loss: 1.0608 - test_loss: 0.845812
Epoch: 125/500 - train_loss: 0.2043 - test_loss: 0.138944
Epoch: 250/500 - train_loss: 0.1695 - test_loss: 0.094950
Epoch: 375/500 - train_loss: 0.1628 - test_loss: 0.119871
Epoch: 499/500 - train_loss: 0.1643 - test_loss: 0.123382
City: Long An  _algo:lstm  -RMSE: 14.860251463654313
City: Long An  _algo:lstm  -RMSE: 10.471725246063391
City: Long An  _algo:lstm  -RMSE: 10.965790738665245
Epoch:  0/380 - train_loss: 0.8461 - test_loss: 0.684423
Epoch: 95/380 - train_loss: 0.1734 - test_loss: 0.111902
Epoch: 190/380 - train_loss: 0.1481 - test_loss: 0.109477
Epoch: 285/380 - train_loss: 0.1405 - test_loss: 0.103806
Epoch: 379/380 - train_loss: 0.1377 - test_loss: 0.100141
City: Long An  _algo:cnn  -RMSE: 9.438583554704522